***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-b5a2  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 30 17:34 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'inforetprojbucket22' 
full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if 'multistream' in b.name:
        paths.append(full_path+b.name)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd
id_title_pairs = parquetFile.select("id","title").rdd
Dict_id_Title = id_title_pairs.collectAsMap()

Building Pickle File of ('doc_id', 'title') for retrieving title easier.

In [11]:
with open('/./home/dataproc/dict_doc_id.pickle', 'wb') as f:
    pickle.dump(Dict_id_Title, f)

In [ ]:
!gsutil cp -r /./home/dataproc/dict_doc_id.pickle

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp_title.py
!ls inverted_index_gcp_body.py
!ls inverted_index_gcp_anchor.py
!ls inverted_index_gcp.py

inverted_index_gcp_title.py
inverted_index_gcp_body.py
inverted_index_gcp_anchor.py
inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp_title.py")
sc.addFile("/home/dataproc/inverted_index_gcp_body.py")
sc.addFile("/home/dataproc/inverted_index_gcp_anchor.py")
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp_title import InvertedIndex as Title_InvertedIndex
from inverted_index_gcp_body import InvertedIndex as Body_InvertedIndex
from inverted_index_gcp_anchor import InvertedIndex as Anchor_InvertedIndex
from inverted_index_gcp import InvertedIndex

**Functions for Indexing**

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  newDict = {}
  returnlist = []
  for token in tokens:
    if token not in all_stopwords:
      if token not in newDict:
        newDict[token] = (id, 1)
      else:
        my_tuple = newDict[token]
        my_list = list(my_tuple)
        my_list[1] = my_list[1]+1
        my_tuple = tuple(my_list)
        newDict[token] = my_tuple
  for token in newDict:
    returnlist.append((token, newDict[token]))      
  return returnlist

def reduce_word_counts(unsorted_pl):
  sorted_list = list(unsorted_pl)
  return sorted(sorted_list, key = lambda x:x[0])

def calculate_df(postings):
  words = postings.map(lambda w : (w[0],len(w[1])))
  return words

def partition_postings_and_write_body(postings):
  GroupedPostings = postings.groupBy(lambda x: token2bucket_id(x[0]))
  MappedPostings = GroupedPostings.map(lambda x: Body_InvertedIndex.write_a_posting_list(x,bucket_name))
  return MappedPostings

def partition_postings_and_write_title(postings):
  GroupedPostings = postings.groupBy(lambda x: token2bucket_id(x[0]))
  MappedPostings = GroupedPostings.map(lambda x: Title_InvertedIndex.write_a_posting_list(x,bucket_name))
  return MappedPostings

def partition_postings_and_write_anchor(postings):
  GroupedPostings = postings.groupBy(lambda x: token2bucket_id(x[0]))
  MappedPostings = GroupedPostings.map(lambda x: Anchor_InvertedIndex.write_a_posting_list(x,bucket_name))
  return MappedPostings


def Calculate_DL(id, text):
  text_counter = 0
  if(len(text) == 1):
    return((id, 1))  
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  for token in tokens:
    if token not in all_stopwords:
      text_counter = text_counter + 1 
  return ((id, text_counter))   

def calculate_term_total(postings):
  def term_total_count(postingList):
    ttSum = 0
    for pair in postingList:
      ttSum+= pair[1]
    return ttSum
    
  # Return the RDD
  return postings.mapValues(term_total_count)

# **Creating Body Index**

In [ ]:
# word counts map
word_counts_body = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
#DL
DL_Array_Body=doc_text_pairs.map(lambda x:Calculate_DL(x[1],x[0]))
DL_Dict_Body = DL_Array_Body.collectAsMap()
#postings
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>50)
w2df_body = calculate_df(postings_filtered_body)
w2df_dict_body = w2df_body.collectAsMap()
#Calculating Term Total
w2termstotal_body = calculate_term_total(postings_filtered_body)
dict_term_total_body = w2termstotal_body.collectAsMap()
# partition posting lists and write out
posting_locs_list_body = partition_postings_and_write_body(postings_filtered_body).collect()

##

# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/body'):
  if not blob.name.endswith("pickle"):
    continue
  print(blob)  
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v) 

<Blob: inforetprojbucket22, postings_gcp/body/0_posting_locs.pickle, 1673220401437359>
<Blob: inforetprojbucket22, postings_gcp/body/100_posting_locs.pickle, 1673220514794591>
<Blob: inforetprojbucket22, postings_gcp/body/101_posting_locs.pickle, 1673220475153607>
<Blob: inforetprojbucket22, postings_gcp/body/102_posting_locs.pickle, 1673220471381199>
<Blob: inforetprojbucket22, postings_gcp/body/103_posting_locs.pickle, 1673220448670804>
<Blob: inforetprojbucket22, postings_gcp/body/104_posting_locs.pickle, 1673220525434400>
<Blob: inforetprojbucket22, postings_gcp/body/105_posting_locs.pickle, 1673220518382799>
<Blob: inforetprojbucket22, postings_gcp/body/106_posting_locs.pickle, 1673220516002251>
<Blob: inforetprojbucket22, postings_gcp/body/107_posting_locs.pickle, 1673220476228890>
<Blob: inforetprojbucket22, postings_gcp/body/108_posting_locs.pickle, 1673220452668428>
<Blob: inforetprojbucket22, postings_gcp/body/109_posting_locs.pickle, 1673220518193038>
<Blob: inforetprojbucke

# Creating Title index

Notice ! If We're Building Small Index - Change bucket_number !!! TO small!!!

In [14]:
# word counts map
word_counts_Title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
#DL
DL_Array_Title=doc_title_pairs.map(lambda x:Calculate_DL(x[1],x[0]))
DL_Dict_Title = DL_Array_Title.collectAsMap()
#postings
postings_Title = word_counts_Title.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
w2df_Title = calculate_df(postings_Title)
w2df_dict_Title = w2df_Title.collectAsMap()
#Calculating Term Total
w2termstotal_Title = calculate_term_total(postings_Title)
dict_term_total_Title = w2termstotal_Title.collectAsMap()
# partition posting lists and write out
posting_locs_list_Title = partition_postings_and_write_title(postings_Title).collect()

##

# collect all posting lists locations into one super-set
super_posting_locs_Title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/title'):
  if not blob.name.endswith("pickle"):
    continue
  print(blob)  
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_Title[k].extend(v)

<Blob: inforetprojbucket22, postings_gcp/title/0_posting_locs.pickle, 1673392003609427>
<Blob: inforetprojbucket22, postings_gcp/title/100_posting_locs.pickle, 1673392011807996>
<Blob: inforetprojbucket22, postings_gcp/title/101_posting_locs.pickle, 1673392011969990>
<Blob: inforetprojbucket22, postings_gcp/title/102_posting_locs.pickle, 1673392012018075>
<Blob: inforetprojbucket22, postings_gcp/title/103_posting_locs.pickle, 1673392005189236>
<Blob: inforetprojbucket22, postings_gcp/title/104_posting_locs.pickle, 1673392006676817>
<Blob: inforetprojbucket22, postings_gcp/title/105_posting_locs.pickle, 1673392006355652>
<Blob: inforetprojbucket22, postings_gcp/title/106_posting_locs.pickle, 1673392011795228>
<Blob: inforetprojbucket22, postings_gcp/title/107_posting_locs.pickle, 1673392005279618>
<Blob: inforetprojbucket22, postings_gcp/title/108_posting_locs.pickle, 1673392012403023>
<Blob: inforetprojbucket22, postings_gcp/title/109_posting_locs.pickle, 1673392012793298>
<Blob: infor

In [41]:
print(postings_Title.take(15))

[('autism', [(25, 1)]), ('abraham', [(307, 1), (1436, 1), (2851, 1)]), ('american', [(309, 1), (615, 1), (659, 1), (771, 1), (863, 1), (869, 1), (966, 1), (1110, 1), (1111, 1), (1112, 1), (1201, 1), (1241, 1), (1488, 1), (1558, 1), (1890, 1), (1902, 1), (1950, 1), (1973, 1), (2011, 1), (2175, 1), (2357, 1), (2386, 1), (2472, 1), (2799, 1), (2856, 1), (3861, 1), (3862, 1), (4166, 1), (4218, 1), (9734, 1), (21673, 1), (21976, 1), (21981, 1), (22050, 1), (22788, 1), (24710, 1), (28265, 1), (29507, 1), (34191, 1), (35922, 1), (37032, 1), (40673, 1)]), ('paris', [(309, 1), (16391, 1), (22989, 1), (24001, 1), (27094, 1)]), ('award', [(316, 1), (3858, 1), (3859, 1), (6725, 1), (13978, 1), (16413, 1), (23658, 1), (24205, 1), (31093, 1), (32769, 1), (40213, 1)]), ('animalia', [(332, 1)]), ('book', [(332, 1), (1726, 1), (2956, 1), (3275, 1), (3778, 1), (3978, 1), (4331, 1), (4332, 1), (4333, 1), (4362, 1), (4363, 1), (4364, 1), (4376, 1), (4377, 1), (4379, 1), (4381, 1), (4382, 1), (4386, 1), (4

# Creating Anchor index

In [ ]:
FlattenedAnchor=doc_anchor_pairs.flatMap(lambda x: x[0])
Anchor=FlattenedAnchor.distinct().groupByKey().mapValues(lambda x: " ".join(x))

# word counts map
word_counts_Anchor = Anchor.flatMap(lambda x: word_count(x[1], x[0]))
postings_anchor = word_counts_Anchor.groupByKey().mapValues(reduce_word_counts)
#DL
def DL(text,id):
  lst_words=[]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  for i in tokens:
    if i not in all_stopwords:
      lst_words.append(i)
  
  return ((id,len(lst_words)))


DL_Array_Anchor=Anchor.map(lambda x:DL(x[1],x[0]))
DL_Dict_Anchor = DL_Array_Anchor.collectAsMap()

#postings_filtered = postings_body.filter(lambda x: len(x[1])>10) #we will filter it on the all corpus (will set it to 50)
w2df_Anchor = calculate_df(postings_anchor)
w2df_dict_Anchor = w2df_Anchor.collectAsMap()

w2termstotal_Anchor = calculate_term_total(postings_anchor)
dict_term_total_anchor = w2termstotal_Anchor.collectAsMap()

posting_locs_list_anchor = partition_postings_and_write_anchor(postings_anchor).collect()

super_posting_locs_Anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/anchor'):
  if not blob.name.endswith("pickle"):
    continue
  print(blob)  
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_Anchor[k].extend(v)

<Blob: inforetprojbucket22, postings_gcp/anchor/0_posting_locs.pickle, 1673221939872239>
<Blob: inforetprojbucket22, postings_gcp/anchor/100_posting_locs.pickle, 1673221946385544>
<Blob: inforetprojbucket22, postings_gcp/anchor/101_posting_locs.pickle, 1673221952644218>
<Blob: inforetprojbucket22, postings_gcp/anchor/102_posting_locs.pickle, 1673221952765149>
<Blob: inforetprojbucket22, postings_gcp/anchor/103_posting_locs.pickle, 1673221947492889>
<Blob: inforetprojbucket22, postings_gcp/anchor/104_posting_locs.pickle, 1673221942877574>
<Blob: inforetprojbucket22, postings_gcp/anchor/105_posting_locs.pickle, 1673221948865516>
<Blob: inforetprojbucket22, postings_gcp/anchor/106_posting_locs.pickle, 1673221952532535>
<Blob: inforetprojbucket22, postings_gcp/anchor/107_posting_locs.pickle, 1673221947815800>
<Blob: inforetprojbucket22, postings_gcp/anchor/108_posting_locs.pickle, 1673221952612297>
<Blob: inforetprojbucket22, postings_gcp/anchor/109_posting_locs.pickle, 1673221949571174>
<

In [15]:
print(posting_locs_list_Title)
# print(posting_locs_list_anchor)
super_posting_locs_anchor = defaultdict(list)
for posting_loc in posting_locs_list_anchor:
    for k, v in posting_loc.items():
        super_posting_locs_anchor[k].extend(v)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123]


NameError: name 'posting_locs_list_anchor' is not defined

# Creating an inverted Index for Body

In [ ]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# print(inverted.posting_locs)
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_dict_body
#DL
inverted_body.DL = DL_Dict_Body
#Term Total
inverted_body.term_total = dict_term_total_body
# write the global stats out
inverted_body.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/body_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 69.0 MiB/ 69.0 MiB]                                                
Operation completed over 1 objects/69.0 MiB.                                     


# Creating an inverted index for Title

In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_Title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_Title
#DL
inverted_title.DL = DL_Dict_Title
#Term Total
inverted_title.term_total = dict_term_total_Title
# write the global stats out
inverted_title.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/title_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][132.9 MiB/132.9 MiB]                                                
Operation completed over 1 objects/132.9 MiB.                                    


# Creating an inverted index for Anchor

In [ ]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_Anchor
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_dict_Anchor
#DL
inverted_anchor.DL = DL_Dict_Anchor
#Term Total
inverted_anchor.term_total = dict_term_total_anchor
# write the global stats out
inverted_anchor.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/anchor_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][166.6 MiB/166.6 MiB]                                                
Operation completed over 1 objects/166.6 MiB.                                    


In [25]:
!gsutil ls -lh $index_dst
# words, pls = zip(*inverted.posting_lists_iter())

 27.74 MiB  2023-01-02T15:52:44Z  gs://inforetprojbucket22/postings_gcp/Multi1_CompleteIndex/anchor/anchor_index.pkl
TOTAL: 1 objects, 29091430 bytes (27.74 MiB)


In [ ]:
gs://{bucket_name}/pr
